In [ ]:
import os
import shlex

BASE_DATA_DIRS = ["videos", "images", "metadata"]
for modal in BASE_DATA_DIRS:
    os.makedirs(os.path.join(f"./data/{modal}/"), exist_ok=True)

# Download all episodes of the specified show

In [ ]:
import os

search_show_names = ["The People of Paradise", "Quest Under Capricorn", "Life On Earth",  "The Private Life Of Plants", "Planet Earth", "The Hunt"]

search_show_name = search_show_names[4]
csv_file_path = "./data/darc-iarchive-wldoc-with-vid-durations.csv"

# video extract params
downloaded_show_name = search_show_name.lower().replace(" ", "-")
image_zfill_mag, fps = 5, 2

# Download the requested show

In [ ]:
!python3 ./scripts/darc_download_shows.py \
    --file_path {csv_file_path} \
    --search_show_name {shlex.quote(search_show_name)}

# Extract Images from Videos at 2 fps

In [ ]:
!python3 ./scripts/darc_extract_video_images.py \
    --show_name {downloaded_show_name} \
    --image_zfill_mag {image_zfill_mag} \
    --fps {fps}

# Image Feature Extraction

In [ ]:
image_folder = f"./data/images/{downloaded_show_name}"

# Image features
vision_model = "google/vit-base-patch16-224-in21k"
bs_vm, nw_vm = 24, 4

!python3 ./scripts/darc_imagefolder_feature_extraction.py \
    --image_folder {image_folder} \
    --vision_model {vision_model} \
    --batch_size {bs_vm} \
    --num_workers {nw_vm}

#python3 ./scripts/darc_imagefolder_feature_extraction.py --image_folder ./data/images/life-on-earth/ --vision_model "google/vit-base-patch16-224-in21k" --batch_size 24 --num_workers 4

# BLIP CAPTIONING

In [ ]:
# Image captions
language_model = "Salesforce/blip2-opt-2.7b"
bs_lm, nw_lm = 6, 6

!python3 ./scripts/darc_imagefolder_textcaption_generation.py \
    --image_folder {image_folder} \
    --language_model {language_model} \
    --batch_size {bs_lm} \
    --num_workers {nw_lm}

# python3 ./scripts/darc_imagefolder_textcaption_generation.py --image_folder ./data/images/life-on-earth/ --language_model "Salesforce/blip2-opt-2.7b" --batch_size 4 --num_workers 4

# OpenAI text embeddings from captions
- While I didnt batch anything to get the LoE data I've implemented and used the openAI API parallel requests script provided by openAI. To use this you have to transfer the data into a jsonl format first (not shown here)

In [ ]:
python3 ./scripts/openai_api_request_parallel_processor.py \
  --requests_filepath "./data/images/life-on-earth/captions_ep_all.jsonl" \
  --request_url https://api.openai.com/v1/embeddings \
  --max_requests_per_minute 1500 \
  --max_tokens_per_minute 6250000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20 \
  --api_key "YOUR-API-KEY"